In [3]:
# Step 1: Create SQLite Database from CSV
# Relationship between Python + SQL : 
# User sends SQL commands thru curson -> cursor executes them in database 
# -> cursor holds the results -> user fetches the results back to python 
import pandas as pd
import sqlite3

# Load data
df = pd.read_csv('diabetic_data_cleaned.csv')

# Create SQLite database
conn = sqlite3.connect('readmission.db')

# Load dataframe into SQL database as a table called 'patients'
df.to_sql('patients', conn, if_exists='replace', index=False)

## METHOD:
# cursor.execute(sql) : run sql command
# cursor.fetchone() : gets 1 row (tuple) from the result # Returns: (value1, value2, value3)
# cursor.fetchone()[0]: get the first value in the tuple
# cursor.fetchall() : gets all rows from results # Returns: [(row1), (row2), (row3), ...]
# cursor.fetchmany(n): get n rows from results
# PRAGMA : SQLite-specific metadata command


# Verify 
cursor = conn.cursor()
cursor.execute("Select count(*) from patients")
total_patients = cursor.fetchone()[0]
print(f"Total patients: {total_patients:,}")

# Show table structure
cursor.execute("PRAGMA table_info(patients)")
columns = cursor.fetchall()
print(f"\nTable has {len(columns)} columns")
conn.close()

Total patients: 101,763

Table has 48 columns


In [4]:
# STEP 2: Write SQL queries to answer Business Questions
import sqlite3

# Connect to database
conn = sqlite3.connect('readmission.db')
cursor = conn.cursor()

# Query 1: overall readmission rate
query1 = '''
select
    count(*) as total_patients,
    sum(case when readmitted = '<30' then 1 else 0 end) as readmitted_patients,
    round(100.0 * sum(case when readmitted = '<30' then 1 else 0 end)/count(*),2) as readmission_rate
from patients;
'''
cursor.execute(query1)
result = cursor.fetchone()
print(f"Total Patients: {result[0]:,}")
print(f"Readmitted within 30 days: {result[1]:,}")
print(f"Readmission Rate: {result[2]}%")
conn.close()

Total Patients: 101,763
Readmitted within 30 days: 11,357
Readmission Rate: 11.16%


In [17]:
# Query 2: Readmission by procedure count
query2 = '''
select
    num_procedures,
    count (*) as total_patients,
    sum(case when readmitted = '<30' then 1 else 0 end) as readmitted_patients,
    round(100.0 * sum(case when readmitted = '<30' then 1 else 0 end)/count(*),2) as readmission_rate
from patients
group by num_procedures
order by num_procedures;
'''
cursor.execute(query2)
result = cursor.fetchall()
print(f"{'Procedures':<12} {'Total':<12} {'Readmitted':<15} {'Rate':<10}")

for row in result:
    num_proc = row[0]
    total = row [1]
    readmitted = row[2]
    rate = row[3]
    print(f"{num_proc:<12} {total:<12,} {readmitted:<15,} {rate:<10}")

print(f"\nKey Finding: Patients with 1 procedure have HIGHEST readmission rate!")


Procedures   Total        Readmitted      Rate      
0            46,652       5,168           11.08     
1            20,741       2,532           12.21     
2            12,716       1,422           11.18     
3            9,443        1,009           10.69     
4            4,180        461             11.03     
5            3,077        290             9.42      
6            4,954        475             9.59      

Key Finding: Patients with 1 procedure have HIGHEST readmission rate!


In [34]:
# Query 3: How does readmission risk increase with num_of_diagnoses?
query3 = '''
select 
    number_diagnoses,
    count(*) as total_patients,
    sum(case when readmitted = '<30' then 1 else 0 end) as readmitted_patients,
    round(100.0 * sum(case when readmitted = '<30' then 1 else 0 end)/count(*),2) as readmission_rate
from patients
group by number_diagnoses
order by number_diagnoses DESC;
'''

cursor.execute(query3)
result = cursor.fetchall()
print(f"{'Diagnoses':<12} {'Total':<12} {'Readmitted':<15} {'Rate':<10}")

for row in result:
    num_diag = row[0]
    total_patients = row[1]
    readmitted = row[2]
    rate = row[3]
    print(f"{num_diag:<12} {total_patients:<12} {readmitted: <15} {rate:<15}")



Diagnoses    Total        Readmitted      Rate      
16           45           4               8.89           
15           10           2               20.0           
14           7            1               14.29          
13           16           3               18.75          
12           9            1               11.11          
11           11           3               27.27          
10           17           3               17.65          
9            49473        6125            12.38          
8            10616        1254            11.81          
7            10393        1119            10.77          
6            10161        1058            10.41          
5            11392        1043            9.16           
4            5536         457             8.26           
3            2835         209             7.37           
2            1023         62              6.06           
1            219          13              5.94           


In [ ]:
# Actionable: "Patients with 6+ diagnoses need intensive case management

In [39]:
# Query 4: Readmission by discharge disposition
query4 = '''
select 
    discharge_disposition_id,
    case 
        when discharge_disposition_id = 1 then 'Home'
        when discharge_disposition_id = 3 then 'Skilled Nursing Facility (SNF)'
        when discharge_disposition_id = 6 then  'Home with home health'
        else 'Other'
        end as discharge_destination,
    count(*) as total_patients,
    sum(case when readmitted = '<30' then 1 else 0 end) as readmitted_patients,
    round(100.0 * sum(case when readmitted = '<30' then 1 else 0 end)/count(*),2) as readmission_rate
from patients
group by discharge_disposition_id
order by discharge_disposition_id ;
'''

cursor.execute(query4)
result = cursor.fetchall()
print(f"{'ID':<5} {'Destination':<35} {'Total':<12} {'Readmitted':<15} {'Rate':<10}")

for row in result:
    id = row[0]
    destination = row[1]
    total_patients = row[2]
    readmitted = row[3]
    rate = row[4]
    print(f"{id:<5} {destination:<35} {total_patients:<12} {readmitted: <15} {rate:<15}")


ID    Destination                         Total        Readmitted      Rate      
1     Home                                60232        5602            9.3            
2     Other                               2128         342             16.07          
3     Skilled Nursing Facility (SNF)      13954        2046            14.66          
4     Other                               815          104             12.76          
5     Other                               1184         247             20.86          
6     Home with home health               12902        1638            12.7           
7     Other                               623          90              14.45          
8     Other                               108          15              13.89          
9     Other                               21           9               42.86          
10    Other                               6            0               0.0            
11    Other                               1642  

In [45]:
# Query 5: High Risk Patient Segment: what types of patients are high-risk? 
# The combo: 1 procedure + SNF + complex 

query5 = '''
select 
    case 
        when num_procedures = 1 and discharge_disposition_id = 3 and number_diagnoses >=6
        then 'HIGH RISK'
        when num_procedures = 1 and number_diagnoses >=6 
        then 'MEDIUM-HIGH'
        when num_procedures = 1
        then 'MEDIUM'
        when num_procedures >=6 and discharge_disposition_id = 3
        then 'MEDIUM-HIGH'
        else 'Standard Risk'
    end as risk_segment,
    count(*) as total_patients,
    sum(case when readmitted = '<30' then 1 else 0 end) as readmitted_patients,
    round(100.0 * sum(case when readmitted = '<30' then 1 else 0 end)/count(*),2) as readmission_rate
from patients
group by risk_segment
order by readmission_rate desc;
'''

cursor.execute(query5)
result = cursor.fetchall()
print(f"{'Risk Segment':<45} {'Total':<12} {'Readmitted':<15} {'Rate':<10}")

for row in result:
    segment = row[0]
    total = row [1]
    readmitted = row[2]
    rate = row[3]
    print(f"{segment:<45} {total:<12} {readmitted:<15} {rate:<10}")


Risk Segment                                  Total        Readmitted      Rate      
HIGH RISK                                     3113         464             14.91     
MEDIUM-HIGH                                   13510        1691            12.52     
Standard Risk                                 80685        8786            10.89     
MEDIUM                                        4455         416             9.34      
